# CCN 2023 Workshop

[Codebook](https://codebook.datajoint.com/hub/?works_context={%22workflow_template%22:%22workshop[…]:%22datajoint-tutorial%22,%22workflow%22:%22ccn2023%22})

This notebooks demonstrates processing calcium imaging scans within DataJoint using CaImAn analysis package.

### How does DataJoint help?

DataJoint is basically a Science Operations framework, like MLOps frameworks. Manages & analyzes data from multimodal neuroscience experiments.

Workflow calcium imaging

 - Organizes metadata from recordings and other resources (e.g. animal-related),
 - Runs the caiman within the workflow without losing track of the input parameters,
 - Stores the results without losing input-output connection.
 

### How is DataJoint related to CaImAn?
Modern calcium imaging analysis, like CaImAn, are pipelines that do some or all of 
MotionCorrection, Segmentation, Classification, Trace extraction tasks.

Workflow-calcium-imaging is designed to run these packages and store their outputs along with animal colony metadata.


### What does the workflow specifically do?

- Stores the relative paths of calcium imaging scans in the database.
- Parses metadata from the scan files and stores in the database.
- Runs the caiman within the workflow, user has to provide the caiman parameters. Parameters are also stored in the database.
- After the analysis, the results are stored in dedicated tables. The pipeline design very much aligns with caiman outputs.
    For instance, the caiman pipeline does MotionCorrection, Segmentation, Classification, and Trace extraction. Results related to these
    tasks are stored in separate tables.
- All metadata and results can be easily accessed via queries, allowing easy downstream analysis.




In [ ]:
import os
import datajoint as dj
import getpass
import numpy as np
from matplotlib import pyplot as plt
import getpass

## Configuration

(Required after installing the workflow on local. Skip this step in the Codebook.)

Enter database credentials. A DataJoint workflow requires a connection to an existing relational database. The connection setup parameters are defined in the dj.config python dictionary.

In [ ]:
dj.config["database.user"] = "tdincer"

#os.makedirs(f"/home/{dj.config['database.user']}/inbox", exist_ok=True)

dj.config["custom"] = {}
dj.config["custom"] = {
    "database.prefix": dj.config["database.user"] + "_ccn2023_",
    "imaging_root_data_dir": "/home/data",
    "processed_root_data_dir": f"/home/{dj.config['database.user']}/outbox",
}

Import schemas of the workflow-calcium-imaging to interact with the database.

In [ ]:
from workflow_calcium_imaging.pipeline import subject, session, imaging, scan, Equipment

Ingesting raw data

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(scan)

Processing scans with CaImAn

In [ ]:
dj.Diagram(scan.Scan) + dj.Diagram(imaging.Processing) + dj.Diagram(imaging.ProcessingParamSet)


Analysis Outputs

In [ ]:
dj.Diagram(imaging.Processing) + \
dj.Diagram(imaging.Curation) + \
dj.Diagram(imaging.MotionCorrection) + \
dj.Diagram(imaging.MaskClassification) + \
dj.Diagram(imaging.Activity) + \
dj.Diagram(imaging.Fluorescence)


## Populating the database

In [ ]:
from workflow_calcium_imaging.pipeline import subject, session, imaging, scan, Equipment
from workflow_calcium_imaging.ingest import ingest_subjects, ingest_sessions  # csv loaders
import csv

In [ ]:
sessions_csv_path = f"/home/{dj.config['database.user']}/sessions.csv"
subject_csv_path = f"/home/{dj.config['database.user']}/subjects.csv"

with open(subject_csv_path, 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["subject", "sex", "subject_birth_date"])
    csv_writer.writerow(["subject0", "M", "2023-02-01"])

with open(sessions_csv_path, 'w', newline='') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["subject","session_dir"])
    csv_writer.writerow(["subject0","subject0/session1/"])
    
ingest_subjects(subject_csv_path=subject_csv_path)
ingest_sessions(session_csv_path=sessions_csv_path)

params_caiman = {
    "decay_time": 0.4,
    "strides": (48, 48),
    "overlaps": (24, 24),
    "max_shifts": (6, 6),
    "max_deviation_rigid": 3,
    "pw_rigid": True,
    "rolling_sum": True,
    "only_init": True,
    "use_cnn": False,
}
    
imaging.ProcessingParamSet.insert_new_params(
    processing_method="caiman", 
    paramset_idx=0,
    params=params_caiman,
    paramset_desc="Calcium imaging analysis with CaImAn using default parameters")


Next, we'll trigger the relevant populate commands.

In [ ]:
session.Session()

In [ ]:
imaging.ProcessingParamSet()

In [ ]:
from workflow_calcium_imaging import process

process.run()
session_key = (session.Session & "subject='subject0'").fetch('KEY')[0]
imaging.ProcessingTask.insert1(
    dict(
        session_key,
        scan_id=0,
        paramset_idx=0,
        processing_output_dir="subject0/session1/",
        task_mode="trigger"
    ),
    skip_duplicates=True
)
process.run()

And then, we'll insert new Curation to trigger ingestion of curated results, followed by the same process.run automation.

In [ ]:
key = (imaging.ProcessingTask & session_key).fetch1('KEY')
imaging.Curation().create1_from_processing_task(key)
process.run()

## Interacting with results

In [ ]:
traces = (imaging.Activity.Trace & "extraction_method='caiman_dff'").fetch('activity_trace')

In [ ]:
imaging.Activity.Trace()

In [ ]:
plt.plot(traces[0]);